# Recurrent neural network

## RNN

In real life, we usually get information continuously or by portions. We can't get all information at once. However, we can still make correct decisions and analyze surroundings because we have *memory*.

Feedforward neural network has only one way of flowing data and the same input always causes the same output. That greatly limits opportunities of these nets in such fields of human life as speech, music, writing and many others. For instance, imagine a task of predicting next letter in a word. Feedforward neural network would fail completely at this task because it can't differ between "t" in "the" and in "tea".

We need to somehow preserve information about previous inputs or states of hidden layers in a neural net. Obviously, simple concatenation of previous ones won't work because the number of these inputs changes and can increase greatly.

### Mathematical idea

Then let's take hidden (or input) layer from the previous step and treat it like a second input of the neural network. We remember that inputs can be represented as vectors or lists in Python, and weights can be represented as tables. From the theory of feedforward neural networks, we can get that after passing of such input list through the table of weights it changes its size (number of the elements) and values of its elements like following:

$$y_i=\sum_k w_{ik} x_k$$

Let's call this operation *multiplication* of a list on a table, which results in a new list. It can be written in mathematical notations as $y=Wx$. Using it, we can finally write that we are taking input from the current step, hidden state from the previous step, multiplicate them, sum by the elements and take activation function from each element:

$$h_t=\sigma(Wx_t+Uh_{t-1})$$

As a result we get formula which completely defines behaviour of the net depending on inputs $x_1,x_2,...x_n$

### Qualitative idea

Taking hidden state from the previous step gives net an information, in theory, about all previous steps since they all influence current output (or just hidden state - our network has one layer). That is why neural net acquires memory and can solve harder problems which were written at the beginning.

![http://colah.github.io](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

This system can have two interpretations. The first one states that we recursively use the previous hidden state as an input. The second one says that we have n neural nets which consecutively pass each other their outputs as inputs. They are equivalent, however, the second one is not used during computation because it is inefficient.

### Keras implementation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#number of examples in a batch 
#gradients are averaged across the batch for better convergence
batch_size=32 
#number of inputs (x1,x2,...) in an example
timesteps=5
#number of elements in the input (length of the input list)
input_dimension=1
#number of neurons in the hidden layer
hidden_dimension=64
#number of elements in the output (length of the output list)
output_dimension=6

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import SimpleRNN

model=Sequential()
model.add(SimpleRNN(hidden_dimension,batch_input_shape=[batch_size,timesteps,input_dimension],dropout=0.5))
model.add(Dense(output_dimension))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
model.summary()

### Vanishing gradient

When we begin training a neural net, we usually assign small values to all weights, which helps to stabilize training. However, if RNN's matrix U, which is responsible for "memory" or transferring previous hidden states to the current neural network, has small values (lower than 1), the impact of previously hidden states is exponentially decreasing moving backward in time. It means that RNN has a low attention span and is unable to remember more than several hidden states backward. As a result, RNN can't keep up with *long* memory problems like audio recognition and language problems. Hopefully, there was invented new recurrent neural network called LSTM, which successfully tackles vanishing gradient problem. We are going to review LSTM in the next chapter.

## LSTM

![http://colah.github.io](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

The heart of LSTM is called cell state, or basically, a vector, which keeps memories. Then LSTM introduces *gates*, which affect cell state. 

The first one called forget gate, which takes previous hidden state and current input, multiplies them by the matrix $W_f$, takes a sigmoid function and multiplies cell state by the output of the sigmoid function $f$. In other words, forget gate tells how many information should be forgotten (multiplication by close to zero numbers).

The second one is input gate, which updates cell state in two stages. Firstly, using the sigmoid function, it calculates what values are going to be updated (1 - updating, 0 - not updating) and stores in the vector $i$. Then a layer of neurons with hyperbolic tangent activation function generates update candidates $C'$. Eventually, these candidates are multiplied elementwise by $i$ and *added* to the cell vector. As a result, cell state changes its value as: $$C_t=C_{t-1}\circ f+C'\circ i$$where $\circ$ denotes elementwise multiplication of vectors (if $c=a\circ b$ then for each k-th element holds $c_k=a_kb_k$).

Finally, we determine current hidden state $h_t$ by taking the sigmoid function from the cell state and multiplying it by the output of tanh (hyperbolic tangent) neuron layer, where the input of the layer is the current input of the network and the previous hidden state:
$$h_t=\sigma(W_ox_t+U_oh_{t-1})\tanh(c_t)$$

The cell state and system of forget/update gates solves gradient vanishing problem. On contrary, it is natural for LSTM to keep track of long dependencies and, therefore, it became widespread in speech recognition, translation and many more.

### One-layer LSTM implementation

We are going to start from the easiest LSTM network, which contains only one recurrent LSTM layer. It can also contain non-recurrent layers as Dense.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM

model=Sequential()
model.add(LSTM(128,batch_input_shape=(50,100,4),dropout=0.5))
model.add(Dense(4,activation='tanh'))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
model.summary()

As in SimpleRNN, we need to set input size, a number of timesteps and batch size. Here we also introduce a method "dropout", which helps in training neural networks. During training, a neuron can be turned off (its output is set to 0) with some probability, which is written after the keyword "dropout" (dropout=0.5).

In the second notebook, you are going to learn more about LSTM training tips and tricks

## Useful links

1. http://colah.github.io/posts/2015-08-Understanding-LSTMs/
2. https://keras.io/layers/recurrent/